In [8]:
from pathlib import Path
from time import sleep
from scripts.data.api_access import refresh_access_token
from scripts.data.secrets.api_access_data import client_secret, app_id, refresh_token
from scripts.data.data_acquisition import ItemDataAcquisition, SellerDataAcquisition, RatingsDataAcquisition

from scripts.data.api_access import call_access_token

In [ ]:
# Para refrescar el access token y volver a escribirlo sobre el archivo access_token.txt
# se puede ejecutar el siguiente código
refresh_access_token(client_secret, app_id, refresh_token);

In [6]:
PATH_DATA = Path("data")

In [2]:
data = ItemDataAcquisition(path_data = PATH_DATA)
# En caso de requerir continuar con la ejecución del proceso
# desde cierta categoría en específico, se puede hacer uso
# del siguiente código para eliminar las categorías ya procesadas
# last_cat_index = prueba.cats[prueba.cats["id"]=="MCO40433"].index[0]
# prueba.cats = prueba.cats[prueba.cats.index >= last_cat_index]

# 41 minutos
data.get_all_items_by_cats(0, 3950)

In [7]:
all_items_data = ItemDataAcquisition.compile_data("data_items", PATH_DATA)
all_items_data.head()

,id,title,condition,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,...,shipping_promise,seller_id,seller_nickname,installments_quantity,installments_amount,installments_rate,installments_currency_id,brand_value_name,location,seller_contact
0,MCO1324022622,Pila Recargables Aa X2 Energizer 2000 Mah,new,MCO21850181,gold_special,https://www.mercadolibre.com.co/pila-recargabl...,buy_it_now,MCO,MCO7279,MCO-CELL_BATTERIES,...,NaN,199769978,OPERADOR TO,36.0,647.47,0.0,COP,Energizer,NaN,NaN
1,MCO1271613775,Pila Energizer Recharge Universal Aaa X 4und,new,MCO22015422,gold_special,https://www.mercadolibre.com.co/pila-energizer...,buy_it_now,MCO,MCO7279,MCO-CELL_BATTERIES,...,NaN,199769978,OPERADOR TO,36.0,1039.64,0.0,COP,Energizer,NaN,NaN
2,MCO1037658202,Audífonos In-ear Inalámbricos Bluetooth F9-5 N...,new,MCO16224063,gold_special,https://www.mercadolibre.com.co/audifonos-in-e...,buy_it_now,MCO,MCO3697,MCO-HEADPHONES,...,NaN,1066063779,JUAN20220131132107,NaN,NaN,NaN,NaN,Shenzhen Yihaotong,NaN,NaN
3,MCO1303242171,Televisor Samsung Cu7000 Crystal Uhd 55 pulgad...,new,MCO23541767,gold_pro,https://www.mercadolibre.com.co/televisor-sams...,buy_it_now,MCO,MCO14903,MCO-TELEVISIONS,...,NaN,1207093176,SAMSUNG COLOMBIA,12.0,135825.00,0.0,COP,Samsung,NaN,NaN
4,MCO889658125,Parlante Jbl Bluetooth Charge 5 Color Black 11...,new,MCO17949052,gold_special,https://www.mercadolibre.com.co/parlante-jbl-b...,buy_it_now,MCO,MCO3691,MCO-SPEAKERS,...,NaN,605176448,EGP_TECNOLOGIA,36.0,16805.56,0.0,COP,JBL,NaN,NaN


In [9]:
data_sellers = SellerDataAcquisition(path_data = PATH_DATA, MAX_THREADS=64)
unique_sellers = all_items_data.seller_id.unique()

data_sellers.get_all_sellers(unique_sellers) # Por 100 1 minuto - Con 6 THREADS 20 segundos - 39 minutos con 64 THREADS para todos los seller
sleep(5)

In [4]:
len(all_items_data.id.unique())

115879

In [10]:
data_rating = RatingsDataAcquisition(path_data = PATH_DATA, MAX_THREADS=64)
items_id = all_items_data.id.sort_values().unique()[:100]

data_rating.get_all_ratings(items_id) # 23 segundos por 100 items con 6 THREADS - 17.5 segundos con 64 THREADS

,one_star,two_star,three_star,four_star,five_star,total_reviews,rating_average,id
0,0,0,0,0,0,0,0.0,MCO1000199903
0,1,1,0,4,20,26,4.6,MCO1000205087
0,0,0,0,1,7,8,4.9,MCO1000205879
0,1,0,3,6,4,14,3.9,MCO1000209952
0,0,0,0,0,0,0,0.0,MCO1000211360
...,...,...,...,...,...,...,...,...
0,0,0,0,1,17,18,4.9,MCO1005887190
0,1,1,0,1,11,14,4.4,MCO1005888259
0,1,1,2,11,98,113,4.8,MCO1005890010
0,0,0,0,0,3,3,5.0,MCO1005890172
